# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [134]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [159]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [8]:
def answer_one():
    spam = spam_data[spam_data['target']==1]
    return len(spam) / len(spam_data) *100
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    longest_token = max(vect.get_feature_names_out()[:], key=len)
    return longest_token
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [379]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    clf = MultinomialNB(alpha=0.1)
    clf.fit(vect.transform(X_train),y_train)

    probabilities = clf.predict_proba(vect.transform(X_test))[:, 1]
    auc_score = roc_auc_score(y_test, probabilities)
    return auc_score
answer_three()

0.991545422134696

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    df = pd.DataFrame(X_train_vectorized.toarray(), columns=vect.get_feature_names_out())
    max_tfidf = df.max().sort_values(ascending=True)
    lowest_tfidf_features = max_tfidf.head(20).sort_index(ascending=True).sort_values(ascending=True)
    highest_tfidf_features = max_tfidf.tail(20).sort_index(ascending=True).sort_values(ascending=False)
    return (lowest_tfidf_features,highest_tfidf_features)
answer_four()

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [380]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_test_vectorized = vect.transform(X_test)
    X_train_vectorized = vect.transform(X_train)
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train_vectorized,y_train)
    probabilities = clf.predict_proba(X_test_vectorized)[:, 1]
    auc_score = roc_auc_score(y_test, probabilities)
    return auc_score
answer_five()

0.9954968337775665

In [372]:
vect = TfidfVectorizer(min_df=3).fit(X_train)
X_train_trfm = vect.transform(X_train)
clfNB = MultinomialNB(alpha=0.1)
clfNB.fit(X_train_trfm, y_train)
X_test_trfm = vect.transform(X_test)
y_pred = clfNB.predict(X_test_trfm)
score = roc_auc_score(y_test, y_pred)

In [373]:
score

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [304]:
def answer_six():
    char_count_no_spam = spam_data['text'][spam_data['target'] == 0].str.len().mean()
    char_count_spam = spam_data['text'][spam_data['target'] == 1].str.len().mean()
    return (char_count_no_spam,char_count_spam)
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [305]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [381]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)

    X_train_vectorized = add_feature(X_train_vectorized,X_train.str.len())
    X_test_vectorized = add_feature(X_test_vectorized,X_test.str.len())
    clf = SVC(C=10000)
    clf.fit(X_train_vectorized,y_train)
    probabilities = clf.decision_function(X_test_vectorized)
    auc_score = roc_auc_score(y_test, probabilities)
    return auc_score
answer_seven()

0.9963202213809143

In [375]:
vect = TfidfVectorizer(min_df=5).fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_test_vectorized = vect.transform(X_test)

X_train_vectorized = add_feature(X_train_vectorized,X_train.str.len())
X_test_vectorized = add_feature(X_test_vectorized,X_test.str.len())
clf = SVC(C=10000)
clf.fit(X_train_vectorized,y_train)
#probabilities = clf.predict((X_test_vectorized))
probabilities = clf.decision_function(X_test_vectorized)

auc_score = roc_auc_score(y_test, probabilities)
auc_score

0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [339]:
def answer_eight():
    avg_digit_no_spam = spam_data['text'][spam_data['target'] == 0].str.count(r'\d').mean()
    avg_digit_spam = spam_data['text'][spam_data['target'] == 1].str.count(r'\d').mean()
    return (avg_digit_no_spam,avg_digit_spam)

answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [382]:
from sklearn.linear_model import LogisticRegression

def answer_nine():

    vect = TfidfVectorizer(min_df=5,ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)

    X_train_vectorized = add_feature(X_train_vectorized,X_train.str.len())
    X_test_vectorized = add_feature(X_test_vectorized,X_test.str.len())

    X_train_vectorized = add_feature(X_train_vectorized,X_train.str.count(r'\d'))
    X_test_vectorized = add_feature(X_test_vectorized,X_test.str.count(r'\d'))
    clf = LogisticRegression(C=100,max_iter=100)
    clf.fit(X_train_vectorized,y_train)
    probabilities = clf.predict_proba((X_test_vectorized))[:, 1]
    auc_score = roc_auc_score(y_test, probabilities)
    return auc_score
answer_nine()

/home/mfb200/Python/applied_data_science/ds_course/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9978566456716975

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [360]:
def answer_ten():
    avg_non_word_no_spam = spam_data['text'][spam_data['target'] == 0].str.count(r'\W').mean()
    avg_non_word_spam = spam_data['text'][spam_data['target'] == 1].str.count(r'\W').mean()
    return(avg_non_word_no_spam,avg_non_word_spam)

answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [399]:
def answer_eleven():
    len(X_train[:2000])
    vect = TfidfVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb').fit(X_train[:2000])
    X_train_vectorized = vect.transform(X_train[:2000])
    X_test_vectorized = vect.transform(X_test)

    X_train_vectorized = add_feature(X_train_vectorized,X_train[:2000].str.len())
    X_test_vectorized = add_feature(X_test_vectorized,X_test.str.len())

    X_train_vectorized = add_feature(X_train_vectorized,X_train[:2000].str.count(r'\d'))
    X_test_vectorized = add_feature(X_test_vectorized,X_test.str.count(r'\d'))

    X_train_vectorized = add_feature(X_train_vectorized,X_train[:2000].str.count(r'\W'))
    X_test_vectorized = add_feature(X_test_vectorized,X_test.str.count(r'\W'))

    clf = LogisticRegression(C=100,max_iter=1000)
    clf.fit(X_train_vectorized,y_train[:2000])
    probabilities = clf.predict_proba((X_test_vectorized))[:, 1]
    auc_score = roc_auc_score(y_test, probabilities)

    feature_names = np.array(vect.get_feature_names_out())
    sorted_coef_index = clf.coef_[0].argsort()

    return (auc_score, feature_names[sorted_coef_index[:10]], feature_names[sorted_coef_index[:-11:-1]])


answer_eleven()

(0.9979542637896203,
 array(['n ', '..', ' i', 'at', 'i ', 'he', 'if ', ' m', 'if', ' lo'],
       dtype=object),
 array(['xt', ' st', 'xt ', 'co', 'ww', 'eply ', 'der', 's ', 'ne', 'ply '],
       dtype=object))